<a href="https://colab.research.google.com/github/jaspresso/WordCloud/blob/master/01_%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%95%BC_%EB%86%80%EC%9E%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 라이브러리 설치
!pip install mysql-connector-python
!pip show sqlalchemy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 32.8 MB/s eta 0:00:00
Name: SQLAlchemy
Version: 2.0.31
Summary: Database Abstraction Library
Home-page: https://www.sqlalchemy.org
Author: Mike Bayer
Author-email: mike_mp@zzzcomputing.com
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: greenlet, typing-extensions
Required-by: bigframes, ipython-sql


In [2]:
# 라이브러리 선언
import pandas as pd
from sqlalchemy import create_engine, inspect
from google.colab import drive

In [3]:
# 구글드라이브 연동
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [4]:
%cd /content/gdrive/MyDrive/Colab Notebooks/analysis_edu/05. 데이터분석 및 데이터 전처리

/content/gdrive/MyDrive/Colab Notebooks/analysis_edu/05. 데이터분석 및 데이터 전처리


In [5]:
# 파일 불러오기
# 구글드라이브 용
googleUrl = "../dataset/customerdata.csv"
# github 용
csDataGitUrl = "https://raw.githubusercontent.com/hyokwan/python-lecture/master/dataset/customerdata.csv"

In [6]:
customerData = pd.read_csv(csDataGitUrl)
customerData

,CUSTID,AVGPRICE,EMI,DEVICECOUNT,PRODUCTAGE,CUSTTYPE
0,A13566,4273.900000,3,6,1.679181,Big-Screen-lover
1,A14219,3642.441950,2,4,2.682023,Sleeping-dog
2,A15312,3653.884565,2,5,3.208202,Sleeping-dog
3,A16605,3713.211107,2,6,0.900000,Early-bird
4,B10634,3391.074215,2,4,2.453656,Sleeping-dog
...,...,...,...,...,...,...
145,Z13253,3678.800000,2,5,2.463670,Big-Screen-lover
146,Z13534,3662.437527,3,6,0.900000,Early-bird
147,Z16428,3516.500000,2,5,2.371301,Big-Screen-lover
148,Z16735,3300.100000,2,5,1.704942,Big-Screen-lover


In [7]:
# 데이터 구조 확인
customerData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   CUSTID       150 non-null    object 
 1   AVGPRICE     150 non-null    float64
 2   EMI          150 non-null    int64  
 3   DEVICECOUNT  150 non-null    int64  
 4   PRODUCTAGE   150 non-null    float64
 5   CUSTTYPE     150 non-null    object 
dtypes: float64(2), int64(2), object(2)
memory usage: 7.2+ KB


In [8]:
# 데이터 설명 (숫자형에 대해서는 기초 통계값을 알려줌)
customerData.describe()

,AVGPRICE,EMI,DEVICECOUNT,PRODUCTAGE
count,150.000000,150.000000,150.000000,150.000000
mean,3432.741108,2.053333,4.973333,1.932601
std,371.571877,0.445510,0.996279,0.807457
min,2390.083736,1.000000,3.000000,0.800000
25%,3214.543122,2.000000,4.000000,1.000000
50%,3408.300000,2.000000,5.000000,2.077000
75%,3664.767001,2.000000,6.000000,2.574525
max,4296.873743,3.000000,6.000000,3.483573


In [9]:
# 데이터 타입 보기
customerData.dtypes

,0
CUSTID,object
AVGPRICE,float64
EMI,int64
DEVICECOUNT,int64
PRODUCTAGE,float64
CUSTTYPE,object


In [10]:
# 테이블 정의서 저장
# csv 파일로 보내기
typeDf = pd.DataFrame(customerData.dtypes)
typeDf.to_csv("./테이블정의서.csv", index=True)

In [11]:
# 1. 원하는 행 조회하기
emiCond = 2
avgPriceCond = 3500

In [12]:
#(예: EMI가 2보다 크고 평균가격이 3500 이상인 데이터만)
csData = customerData.loc[ (customerData.EMI > emiCond) &
                  (customerData.AVGPRICE >= avgPriceCond)]

In [13]:
csData

,CUSTID,AVGPRICE,EMI,DEVICECOUNT,PRODUCTAGE,CUSTTYPE
0,A13566,4273.900000,3,6,1.679181,Big-Screen-lover
33,G16437,3787.328898,3,6,1.000000,Early-bird
40,H16326,3988.935304,3,6,0.900000,Early-bird
51,J10809,3676.135627,3,6,0.800000,Early-bird
67,N13273,3839.694732,3,6,1.100000,Early-bird
95,R16364,3592.353540,3,6,0.900000,Early-bird
107,S17396,3895.200000,3,6,2.055707,Big-Screen-lover
113,T11473,4165.700000,3,6,1.780674,Big-Screen-lover
126,V18566,3527.701867,3,6,0.900000,Early-bird
130,W13404,4296.873743,3,6,1.100000,Early-bird


In [14]:
# 0~2 index까지 데이터 추출
customerData.loc[0:2]

# loc는 Pandas 라이브러리에서 DataFrame 객체의 특정 행과 열을 라벨로 접근하여
#데이터를 선택하거나 수정할 수 있는 메서드

# 주요 특징
# 1) loc는 라벨 기반 인덱싱을 사용.
# 2) 행과 열을 콤마(,)로 구분하여 지정.
# 3) 조건을 사용하여 특정 행을 선택할 수 있다.
# 4) 데이터의 업데이트에도 사용할 수 있다.


,CUSTID,AVGPRICE,EMI,DEVICECOUNT,PRODUCTAGE,CUSTTYPE
0,A13566,4273.900000,3,6,1.679181,Big-Screen-lover
1,A14219,3642.441950,2,4,2.682023,Sleeping-dog
2,A15312,3653.884565,2,5,3.208202,Sleeping-dog


In [15]:
# 전체 데이터 프레임의 크기 확인
customerData.shape

(150, 6)

In [16]:
# 0번째 데이터 프레임 크기 확인
customerData.loc[0].shape

(6,)

In [17]:
csData.shape

(11, 6)

In [18]:
# 데이터프레임 컬럼정보를 시연
customerData.columns

Index(['CUSTID', 'AVGPRICE', 'EMI', 'DEVICECOUNT', 'PRODUCTAGE', 'CUSTTYPE'], dtype='object')

In [19]:
# 문자열 함수 소문자
customerData.columns.str.lower()

Index(['custid', 'avgprice', 'emi', 'devicecount', 'productage', 'custtype'], dtype='object')

In [20]:
# 데이터프레임 컬럼정보 수정 * 문자열함수를 사용하려면 .str을 df에 써야 한다.
customerData.columns = customerData.columns.str.lower()


In [21]:
customerData.columns

Index(['custid', 'avgprice', 'emi', 'devicecount', 'productage', 'custtype'], dtype='object')

In [22]:
customerData.columns = customerData.columns.str.upper()

In [23]:
customerData.loc[ (customerData.EMI > emiCond) &
                  (customerData.AVGPRICE >= avgPriceCond)]

,CUSTID,AVGPRICE,EMI,DEVICECOUNT,PRODUCTAGE,CUSTTYPE
0,A13566,4273.900000,3,6,1.679181,Big-Screen-lover
33,G16437,3787.328898,3,6,1.000000,Early-bird
40,H16326,3988.935304,3,6,0.900000,Early-bird
51,J10809,3676.135627,3,6,0.800000,Early-bird
67,N13273,3839.694732,3,6,1.100000,Early-bird
95,R16364,3592.353540,3,6,0.900000,Early-bird
107,S17396,3895.200000,3,6,2.055707,Big-Screen-lover
113,T11473,4165.700000,3,6,1.780674,Big-Screen-lover
126,V18566,3527.701867,3,6,0.900000,Early-bird
130,W13404,4296.873743,3,6,1.100000,Early-bird


In [24]:
# 갯수파악
customerData.loc[ (customerData.EMI > emiCond) &
                  (customerData.AVGPRICE >= avgPriceCond)].shape

(11, 6)

In [25]:
# 검증 코드(위와 반대되는 로직)도 반드시 제공, 갯수도 파악
customerData.loc[ (customerData.EMI <= emiCond) |
                  (customerData.AVGPRICE < avgPriceCond) ]

,CUSTID,AVGPRICE,EMI,DEVICECOUNT,PRODUCTAGE,CUSTTYPE
1,A14219,3642.441950,2,4,2.682023,Sleeping-dog
2,A15312,3653.884565,2,5,3.208202,Sleeping-dog
3,A16605,3713.211107,2,6,0.900000,Early-bird
4,B10634,3391.074215,2,4,2.453656,Sleeping-dog
5,B16849,3755.263391,2,4,2.786665,Sleeping-dog
...,...,...,...,...,...,...
144,Z12772,3029.600000,2,4,2.093035,Big-Screen-lover
145,Z13253,3678.800000,2,5,2.463670,Big-Screen-lover
147,Z16428,3516.500000,2,5,2.371301,Big-Screen-lover
148,Z16735,3300.100000,2,5,1.704942,Big-Screen-lover


In [26]:
# 갯수파악
customerData.loc[ (customerData.EMI <= emiCond) |
                  (customerData.AVGPRICE < avgPriceCond) ].shape

(139, 6)

In [27]:
# 2. 원하는 행 조회하기 (in not in 조건)
# 무이자 할부가 2 또는 3인 경우
emiMultiCond = [2,3]

In [28]:
# ~ 은 not을 의미, 즉 무이자 할부 2 또는 3이 아닌 모든 데이터
customerData.loc[ ~ customerData.EMI.isin( emiMultiCond ) ]

,CUSTID,AVGPRICE,EMI,DEVICECOUNT,PRODUCTAGE,CUSTTYPE
11,C18622,3246.000000,1,5,1.891624,Big-Screen-lover
46,I14835,3374.091339,1,4,2.749678,Sleeping-dog
47,I17056,2963.370122,1,3,2.139196,Sleeping-dog
56,K10712,3419.462596,1,4,2.825790,Sleeping-dog
57,K13300,3159.839204,1,3,1.960833,Sleeping-dog
65,M16452,3353.537290,1,4,1.940357,Sleeping-dog
68,N14862,2390.083736,1,6,1.000000,Early-bird
72,O13037,3398.216334,1,4,2.994395,Sleeping-dog
73,O13586,2980.229239,1,3,2.038603,Sleeping-dog
86,Q16449,3048.308860,1,3,2.369758,Sleeping-dog


In [29]:
# 상위 2개의 행 반환
customerData.loc[ ~ customerData.EMI.isin( emiMultiCond )  ].head(2)

,CUSTID,AVGPRICE,EMI,DEVICECOUNT,PRODUCTAGE,CUSTTYPE
11,C18622,3246.000000,1,5,1.891624,Big-Screen-lover
46,I14835,3374.091339,1,4,2.749678,Sleeping-dog


In [30]:
# 데이터 타입 확인
customerData.dtypes

,0
CUSTID,object
AVGPRICE,float64
EMI,int64
DEVICECOUNT,int64
PRODUCTAGE,float64
CUSTTYPE,object


In [31]:
# 3.원하는 행 조회하기 (컬럼 값 세부 조건)
customerData.loc [customerData.CUSTID.astype(str).str[0] == "A"]

,CUSTID,AVGPRICE,EMI,DEVICECOUNT,PRODUCTAGE,CUSTTYPE
0,A13566,4273.900000,3,6,1.679181,Big-Screen-lover
1,A14219,3642.441950,2,4,2.682023,Sleeping-dog
2,A15312,3653.884565,2,5,3.208202,Sleeping-dog
3,A16605,3713.211107,2,6,0.900000,Early-bird


In [32]:
# 4. 컬럼명 조작방법
# 데이터프레임 컬럼 정보를 시연!
customerData.columns

Index(['CUSTID', 'AVGPRICE', 'EMI', 'DEVICECOUNT', 'PRODUCTAGE', 'CUSTTYPE'], dtype='object')

In [33]:
# 데이터프레임 컬럼정보 수정 * 문자열함수를 사용하려면 .str을 df에서는 써야한다!
customerData.columns = customerData.columns.str.upper()

In [34]:
## 전체 데이터 프레임의 크기 확인용
customerData.shape

(150, 6)

In [ ]:
### 2. 데이터 타입변환 (astype)

In [35]:
# EMI 열의 데이터 타입을 문자열(str)로 변환
customerData.EMI = customerData.EMI.astype(str)

In [36]:
# 데이터 타입 확인
customerData.dtypes

,0
CUSTID,object
AVGPRICE,float64
EMI,object
DEVICECOUNT,int64
PRODUCTAGE,float64
CUSTTYPE,object


In [37]:
# customerData 데이터프레임의 모든 열 이름을 리스트로 변환하여 columnNameList에 저장
columnNameList = list( customerData.columns )

In [38]:
# 숫자를 문자로 변환 (이유 : MySQL => Orcle 이관, 즉, 종류가 다른 DB데이터의
# 데이터 타입이 모두 다르기 때문에 통일해주기 위해 모두 문자로 바꿔서 처리)

In [39]:
# columnNameList의 각 열 이름을 순회
for i in range(0, len(columnNameList)):
    # 현재 열(columnNameList[i])의 데이터를 문자열(str)로 변환하여 다시 해당 열에 저장
    customerData[ columnNameList[i] ] = customerData[ columnNameList[i] ].astype(str)

In [40]:
# EMI 열의 값이 "2"인 행들을 선택하여 반환
customerData.loc[customerData.EMI == "2"]

,CUSTID,AVGPRICE,EMI,DEVICECOUNT,PRODUCTAGE,CUSTTYPE
1,A14219,3642.441950493661,2,4,2.68202302093618,Sleeping-dog
2,A15312,3653.88456537549,2,5,3.2082015713118,Sleeping-dog
3,A16605,3713.211106711161,2,6,0.9,Early-bird
4,B10634,3391.07421460378,2,4,2.45365640323609,Sleeping-dog
5,B16849,3755.26339111697,2,4,2.78666516346857,Sleeping-dog
...,...,...,...,...,...,...
144,Z12772,3029.6,2,4,2.09303462123498,Big-Screen-lover
145,Z13253,3678.8,2,5,2.46367035079282,Big-Screen-lover
147,Z16428,3516.5,2,5,2.37130050714128,Big-Screen-lover
148,Z16735,3300.1,2,5,1.70494219968095,Big-Screen-lover


In [41]:
customerData.EMI.dtype

dtype('O')

In [42]:
# EMI 열의 모든 값을 문자열(str) 타입으로 변환하여 다시 EMI 열에 저장
customerData["EMI"] = customerData["EMI"].astype(str)

In [ ]:
customerData["EMI"].dtype

dtype('O')

In [43]:
# EMI_NEW라는 새로운 컬럼생성
customerData["EMI_NEW"] = customerData["EMI"].astype(str)

In [44]:
targetColumns = ["AVGPRICE"]

In [45]:
# 컬럼이름으로 전체행 검사 및 값 조회
customerData.loc[ : , targetColumns ]

,AVGPRICE
0,4273.9
1,3642.441950493661
2,3653.88456537549
3,3713.211106711161
4,3391.07421460378
...,...
145,3678.8
146,3662.43752684176
147,3516.5
148,3300.1


In [46]:
# 첫 번째 행의 값을 배열로 반환
customerData.loc[ : , targetColumns ].values[0]

array(['4273.9'], dtype=object)

In [47]:
# iloc 인덱서를 사용해 첫 번째와 네 번째 열을 선택하여 DataFrame을 만든 후, 상위 2개의 행을 반환
# index 기법은 유지보수에 편하기는 하나 value 기법에 비해서는 가독성이 떨어짐.
customerData.iloc[ : , [0,3]].head(2)

,CUSTID,DEVICECOUNT
0,A13566,6
1,A14219,4


In [48]:
# 'AVGPRICE' 열의 위치 인덱스를 찾아서 avgpriceColNo 변수에 저장
avgpriceColNo = customerData.columns.get_loc("AVGPRICE")

# 'EMI' 열의 위치 인덱스를 찾아서 emiColNo 변수에 저장
emiColNo = customerData.columns.get_loc("EMI")

In [49]:
# 위치 인덱스가 avgpriceColNo와 emiColNo인 열들을 선택하여 DataFrame을 만든 후, 상위 2개의 행을 반환
customerData.iloc[ : , [avgpriceColNo,emiColNo]].head(2)

,AVGPRICE,EMI
0,4273.9,3
1,3642.441950493661,2


In [50]:
### 3. 인덱스 초기화
targetColumns = ["AVGPRICE", "EMI"]

In [51]:
customerDataIndex = customerData.loc[ customerData.AVGPRICE.astype(float) < 3000,
                 targetColumns ].reset_index(drop=True)
                 # .shape, .index

In [52]:
customerDataIndex

,AVGPRICE,EMI
0,2991.0147029320697,2
1,2744.8158811217604,2
2,2873.10220220129,2
3,2963.3701215062,1
4,2804.26715745362,2
5,2811.26593163706,3
6,2650.9,2
7,2390.08373567855,1
8,2980.22923915426,1
9,2972.10819840076,2


In [53]:
paramList = ["HOST","PORT","USERID","USERPASS","DBNAME"]
paramaValue = ["13.124.134.168","3306","kopouser","kopouser","kopo"]
paramDf = pd.DataFrame( zip(paramList, paramaValue),
             columns = ["PRAM_NAME","PARAM_VALUE"] )
paramDf

,PRAM_NAME,PARAM_VALUE
0,HOST,13.124.134.168
1,PORT,3306
2,USERID,kopouser
3,USERPASS,kopouser
4,DBNAME,kopo


In [54]:
ipName = "HOST"

In [55]:
targetColumn = ["PARAM_VALUE"]

In [56]:
# values[0][0] : 0번째 인덱스의 0번째 실제값
paramDf.loc[ paramDf.PRAM_NAME == ipName, targetColumn].values[0][0]

'13.124.134.168'

In [57]:
### 핵심 데이터 조회를 통해 값을 받아 처리
### 데이터베이스 저장(MySQL)
paramDf = pd.read_csv("./paramData.txt")
ipName = "HOST"
portName = "PORT"
userIdName = "USERID"
userPassName = "USERPASS"
dbName = "DBNAME"
targetColum = ["PARAM_VALUE"]
dbIp = paramDf.loc[ paramDf.PARAM_NAME == ipName,targetColum].values[0][0]
dbPort = paramDf.loc[ paramDf.PARAM_NAME == portName,targetColum].values[0][0]
dbUserName = paramDf.loc[ paramDf.PARAM_NAME == userIdName,targetColum].values[0][0]
dbUserPass = paramDf.loc[ paramDf.PARAM_NAME == userPassName,targetColum].values[0][0]
dbName = paramDf.loc[ paramDf.PARAM_NAME == dbName,targetColum].values[0][0]


In [58]:
# DB 데이터 저장 (MySQL)
# MySQL 데이터베이스 연결 정보 설정 (클라우드)
host = dbIp
port = dbPort
user = dbUserName
password = dbUserPass
database = dbName

# SQLAlchemy 엔진 생성
engine = create_engine(f'mysql+mysqlconnector://{user}:{password}@{host}:{port}/{database}')

# 데이터프레임을 MySQL 데이터베이스의 테이블에 저장
table_name = 'paramdf_0731_final'
paramDf.to_sql(name=table_name, con=engine, if_exists='replace', index=False)

5

# 8월 1일 강의


In [59]:
# 문제] kopo_customerdata.csv에서 statename, gender 컬럼 첫 5개행(0~4)을 조회
csDataLocal = pd.read_csv("../dataset/kopo_customerdata.csv")
csDataGitUrl = "https://raw.githubusercontent.com/hyokwan/python-lecture/master/dataset/kopo_customerdata.csv"

In [60]:
kopo_customerData = pd.read_csv(csDataGitUrl)
targetColumns = ["STATENAME", "GENDER"]
kopo_customerData.loc[ : , targetColumns] # 전체 데이터 조회

,STATENAME,GENDER
0,State2,Male
1,State2,Male
2,State2,Female
3,State2,Male
4,State2,Male
...,...,...
299995,State1,Male
299996,State1,Male
299997,State1,Female
299998,State1,Male


In [61]:
# 0~4까지 index의 행까지 조회
kopo_customerData.loc[ 0:4 , targetColumns]

,STATENAME,GENDER
0,State2,Male
1,State2,Male
2,State2,Female
3,State2,Male
4,State2,Male


In [62]:
# 데이터 기본조회
# 데이터프레임명.loc[ 조건 ]
# 단 조건은 데이터프레임명.컬럼명에 대한 조건 기입해야함
# 또한 각 컬럼값의 문자열 함수를 사용하고 싶은 경우
# 데이터프레임명.str.upper() 형태로 사용 가능 (예: .str.replace("/","-") 등)
kopo_customerData.loc[ kopo_customerData.GENDER.str.upper()=="FEMALE", targetColumns]

,STATENAME,GENDER
2,State2,Female
6,State2,Female
7,State2,Female
11,State2,Female
12,State2,Female
...,...,...
299940,State1,Female
299955,State1,Female
299962,State1,Female
299982,State1,Female


In [63]:
kopo_customerData = pd.read_csv(csDataGitUrl)
targetColumns = ["STATENAME","GENDER"]
kopo_customerData.loc[ 0:4, targetColumns]

,STATENAME,GENDER
0,State2,Male
1,State2,Male
2,State2,Female
3,State2,Male
4,State2,Male


In [67]:
# 데이터 조작 ( ex:PRODUCTAGE 조작 )
import numpy as np

# 전에 문자열로 모두 바꾸었기 때문에 .astype(float)을 더 써줘야함.
customerData['PRODUCTAGE_NEW']=np.where( customerData.PRODUCTAGE.astype(float) < 1, 1, customerData.PRODUCTAGE )

In [68]:
customerData

,CUSTID,AVGPRICE,EMI,DEVICECOUNT,PRODUCTAGE,CUSTTYPE,EMI_NEW,PRODUCTAGE_NEW
0,A13566,4273.9,3,6,1.67918084803969,Big-Screen-lover,3,1.67918084803969
1,A14219,3642.441950493661,2,4,2.68202302093618,Sleeping-dog,2,2.68202302093618
2,A15312,3653.88456537549,2,5,3.2082015713118,Sleeping-dog,2,3.2082015713118
3,A16605,3713.211106711161,2,6,0.9,Early-bird,2,1
4,B10634,3391.07421460378,2,4,2.45365640323609,Sleeping-dog,2,2.45365640323609
...,...,...,...,...,...,...,...,...
145,Z13253,3678.8,2,5,2.46367035079282,Big-Screen-lover,2,2.46367035079282
146,Z13534,3662.43752684176,3,6,0.9,Early-bird,3,1
147,Z16428,3516.5,2,5,2.37130050714128,Big-Screen-lover,2,2.37130050714128
148,Z16735,3300.1,2,5,1.70494219968095,Big-Screen-lover,2,1.70494219968095


In [69]:
# 검증로직(사실은 검증로직이 더 중요함.)
customerData.loc[ (customerData.PRODUCTAGE.astype(float) < 1 ) &
                 (customerData.PRODUCTAGE != 1)]

,CUSTID,AVGPRICE,EMI,DEVICECOUNT,PRODUCTAGE,CUSTTYPE,EMI_NEW,PRODUCTAGE_NEW
3,A16605,3713.211106711161,2,6,0.9,Early-bird,2,1
6,B18816,3213.91144567674,2,6,0.9,Early-bird,2,1
13,D11795,2991.0147029320697,2,6,0.8,Early-bird,2,1
23,E19404,3764.94041410596,2,6,0.9,Early-bird,2,1
24,F10222,3191.06582222132,2,6,0.9,Early-bird,2,1
30,G11453,2744.8158811217604,2,6,0.9,Early-bird,2,1
37,G19717,2873.10220220129,2,6,0.8,Early-bird,2,1
38,H10188,4020.24516165085,2,6,0.9,Early-bird,2,1
40,H16326,3988.9353035127,3,6,0.9,Early-bird,3,1
44,I11601,3437.81838816903,2,6,0.9,Early-bird,2,1


In [70]:
# 검증로직
customerData.loc[ (customerData.PRODUCTAGE.astype(float) >= 1) &
                  (customerData.PRODUCTAGE_NEW != customerData.PRODUCTAGE) ]

,CUSTID,AVGPRICE,EMI,DEVICECOUNT,PRODUCTAGE,CUSTTYPE,EMI_NEW,PRODUCTAGE_NEW


In [71]:
# customerData_new < 1 = 1
customerData["PRODUCTAGE_NEW"]=np.where(  customerData.PRODUCTAGE.astype(float) < 1, 1,
             np.where(  customerData.PRODUCTAGE.astype(float) < 2, 2,
                        customerData.PRODUCTAGE))

In [72]:
## 조작하기 (함수활용 전)
inValue = 1.5
outValue = 1
# 1보다 작으면 1, 2보다 작으면 2, 이외에는 3
if inValue < 1:
    outValue = 1
elif inValue < 2:
    outValue = 2
else:
    outValue = 3
outValue

2

In [73]:
# 함수화 하기
def productAgeCal( inValue = 0.5 ):
  """
  정의: 값을 입력받아 1보다 작으면 1, 2보다 작으면 2, 이외에는 3값을 리턴
  """
  # 기존에 데이터를 모두 문자로 바꾸었기에 변환 해줘야함
  inValue = float(inValue)
  outValue = 1
  if inValue < 1:
      outValue = 1
  elif inValue < 2:
      outValue = 2
  else:
      outValue = 3
  return outValue


In [74]:
# 함수를 적용한 변수
customerData["PRODUCTAGE_NEW"] = customerData.PRODUCTAGE.apply(productAgeCal)

In [75]:
# 출력
customerData["PRODUCTAGE_NEW"]

,PRODUCTAGE_NEW
0,2
1,3
2,3
3,1
4,3
5,3
6,1
7,3
8,3
9,3


In [76]:
# loc 활용 데이터 조작하기
### LOC 활용 데이터 조작 [특정 데이터만 선택 후 조작 시 ]
# 데이터프레임명.loc[조건식, "컬럼"] = 할당 값
customerData.loc[ customerData.PRODUCTAGE.astype(float) < 1, "PRODUCTAGE_NEW" ] = 1
customerData.loc[ (customerData.PRODUCTAGE.astype(float) >= 1) &
                  (customerData.PRODUCTAGE.astype(float) < 2), "PRODUCTAGE_NEW" ] = 2
customerData.loc[ customerData.PRODUCTAGE.astype(float) >= 2, "PRODUCTAGE_NEW" ] = 3

In [101]:
targetColums = ["PRODUCTAGE" ]
# 조건에 맞는 데이터 조회
customerData.loc[ customerData.PRODUCTAGE.astype(float) < 1, targetColums ]
# 조건에 맞는 특정 값 가져오기
customerData.loc[ customerData.PRODUCTAGE.astype(float) < 1, targetColums ].values[0][0]

'0.9'

In [102]:
# A13566 고객의 [첫 번째 행], [첫 번째 열의 값]을 선택해서 평균 가격을 추출
csA13566avgPrice = customerData.loc[ customerData.CUSTID =="A13566", targetColums ].values[0][0]

In [103]:
csA13566avgPrice

'1.67918084803969'

In [113]:
customerData

,CUSTID,AVGPRICE,EMI,DEVICECOUNT,PRODUCTAGE,CUSTTYPE,EMI_NEW,PRODUCTAGE_NEW,NEW_COLUMN,CUST_SEG,CUSTTYPE_NO,PRODUCT_AGE_NEW
0,A13566,5000,3,6,1.67918084803969,Big-Screen-lover,3,2,13566_3,high,1,2
1,A14219,3642.441950493661,2,4,2.68202302093618,Sleeping-dog,2,3,14219_2,mid,2,3
2,A15312,3653.88456537549,2,5,3.2082015713118,Sleeping-dog,2,3,15312_2,mid,2,5
3,A16605,3713.211106711161,2,6,0.9,Early-bird,2,1,16605_2,mid,3,1
4,B10634,3391.07421460378,2,4,2.45365640323609,Sleeping-dog,2,3,10634_2,mid,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...
145,Z13253,3678.8,2,5,2.46367035079282,Big-Screen-lover,2,3,13253_2,mid,1,3
146,Z13534,3662.43752684176,3,6,0.9,Early-bird,3,1,13534_3,mid,3,1
147,Z16428,3516.5,2,5,2.37130050714128,Big-Screen-lover,2,3,16428_2,mid,1,3
148,Z16735,3300.1,2,5,1.70494219968095,Big-Screen-lover,2,2,16735_2,mid,1,2


In [125]:
# 컬럼간 데이터 연산 (조작) : CUSTID A13566에서 A를 제거하고 EMI의 3을 추출해서
# 131566_3으로 만들기 위한 방법
customerData["NEW_COLUMN"] = customerData.CUSTID.str[1:] + "_" + customerData.EMI
customerData["NEW_COLUMN"]

,NEW_COLUMN
0,13566_3
1,14219_2
2,15312_2
3,16605_2
4,10634_2
5,16849_2
6,18816_2
7,10639_2
8,12717_2
9,15050_2


In [126]:
### 컬럼 생성 및 중복값 저장
customerData["NEW_COLUMN2"] = 111
customerData

,CUSTID,AVGPRICE,EMI,DEVICECOUNT,PRODUCTAGE,CUSTTYPE,EMI_NEW,PRODUCTAGE_NEW,NEW_COLUMN,CUST_SEG,CUSTTYPE_NO,PRODUCT_AGE_NEW,NEW_COLUMN2
0,A13566,5000,3,6,1.67918084803969,Big-Screen-lover,3,2,13566_3,high,1,2,111
1,A14219,3642.441950493661,2,4,2.68202302093618,Sleeping-dog,2,3,14219_2,mid,2,3,111
2,A15312,3653.88456537549,2,5,3.2082015713118,Sleeping-dog,2,3,15312_2,mid,2,5,111
3,A16605,3713.211106711161,2,6,0.9,Early-bird,2,1,16605_2,mid,3,1,111
4,B10634,3391.07421460378,2,4,2.45365640323609,Sleeping-dog,2,3,10634_2,mid,2,3,111
...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,Z13253,3678.8,2,5,2.46367035079282,Big-Screen-lover,2,3,13253_2,mid,1,3,111
146,Z13534,3662.43752684176,3,6,0.9,Early-bird,3,1,13534_3,mid,3,1,111
147,Z16428,3516.5,2,5,2.37130050714128,Big-Screen-lover,2,3,16428_2,mid,1,3,111
148,Z16735,3300.1,2,5,1.70494219968095,Big-Screen-lover,2,2,16735_2,mid,1,2,111


In [127]:
# 컬럼 삭제
customerData = customerData.drop(columns= ["NEW_COLUMN2"] )
customerData

,CUSTID,AVGPRICE,EMI,DEVICECOUNT,PRODUCTAGE,CUSTTYPE,EMI_NEW,PRODUCTAGE_NEW,NEW_COLUMN,CUST_SEG,CUSTTYPE_NO,PRODUCT_AGE_NEW
0,A13566,5000,3,6,1.67918084803969,Big-Screen-lover,3,2,13566_3,high,1,2
1,A14219,3642.441950493661,2,4,2.68202302093618,Sleeping-dog,2,3,14219_2,mid,2,3
2,A15312,3653.88456537549,2,5,3.2082015713118,Sleeping-dog,2,3,15312_2,mid,2,5
3,A16605,3713.211106711161,2,6,0.9,Early-bird,2,1,16605_2,mid,3,1
4,B10634,3391.07421460378,2,4,2.45365640323609,Sleeping-dog,2,3,10634_2,mid,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...
145,Z13253,3678.8,2,5,2.46367035079282,Big-Screen-lover,2,3,13253_2,mid,1,3
146,Z13534,3662.43752684176,3,6,0.9,Early-bird,3,1,13534_3,mid,3,1
147,Z16428,3516.5,2,5,2.37130050714128,Big-Screen-lover,2,3,16428_2,mid,1,3
148,Z16735,3300.1,2,5,1.70494219968095,Big-Screen-lover,2,2,16735_2,mid,1,2


In [128]:
### 검증
# AVGPRICE < 3000 - low  3000 미만은 전부 low로 변경했다
                   # 3000 미만에서 low 여야 한다
                   # 3000 이상에서는 low 가 존재하면 안된다
# AVGPRICE 3000>=  400 <= mid
# 4000>=  high
customerData["CUST_SEG"] = np.where(customerData.AVGPRICE.astype(float) < 3000, "low",
         np.where(customerData.AVGPRICE.astype(float) < 4000, "mid", "high"))
customerData["CUST_SEG"]

,CUST_SEG
0,high
1,mid
2,mid
3,mid
4,mid
5,mid
6,mid
7,mid
8,mid
9,high


In [129]:
# AVGPRICE가 3000보다 작고, CUST_SEG가 "low"가 아닌 고객 데이터를 선택
customerData.loc[(customerData['AVGPRICE'].astype(float) < 3000) &
                                 (customerData['CUST_SEG'] != "low")]
customerData

,CUSTID,AVGPRICE,EMI,DEVICECOUNT,PRODUCTAGE,CUSTTYPE,EMI_NEW,PRODUCTAGE_NEW,NEW_COLUMN,CUST_SEG,CUSTTYPE_NO,PRODUCT_AGE_NEW
0,A13566,5000,3,6,1.67918084803969,Big-Screen-lover,3,2,13566_3,high,1,2
1,A14219,3642.441950493661,2,4,2.68202302093618,Sleeping-dog,2,3,14219_2,mid,2,3
2,A15312,3653.88456537549,2,5,3.2082015713118,Sleeping-dog,2,3,15312_2,mid,2,5
3,A16605,3713.211106711161,2,6,0.9,Early-bird,2,1,16605_2,mid,3,1
4,B10634,3391.07421460378,2,4,2.45365640323609,Sleeping-dog,2,3,10634_2,mid,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...
145,Z13253,3678.8,2,5,2.46367035079282,Big-Screen-lover,2,3,13253_2,mid,1,3
146,Z13534,3662.43752684176,3,6,0.9,Early-bird,3,1,13534_3,mid,3,1
147,Z16428,3516.5,2,5,2.37130050714128,Big-Screen-lover,2,3,16428_2,mid,1,3
148,Z16735,3300.1,2,5,1.70494219968095,Big-Screen-lover,2,2,16735_2,mid,1,2


In [130]:
# 연습문제] customerData 변수에서 CUSTID 가 A13566인 고객의 AVGPRICE 값을 5000으로 변경
customerData.loc[customerData['CUSTID'] == 'A13566', 'AVGPRICE'] = 5000
customerData

,CUSTID,AVGPRICE,EMI,DEVICECOUNT,PRODUCTAGE,CUSTTYPE,EMI_NEW,PRODUCTAGE_NEW,NEW_COLUMN,CUST_SEG,CUSTTYPE_NO,PRODUCT_AGE_NEW
0,A13566,5000,3,6,1.67918084803969,Big-Screen-lover,3,2,13566_3,high,1,2
1,A14219,3642.441950493661,2,4,2.68202302093618,Sleeping-dog,2,3,14219_2,mid,2,3
2,A15312,3653.88456537549,2,5,3.2082015713118,Sleeping-dog,2,3,15312_2,mid,2,5
3,A16605,3713.211106711161,2,6,0.9,Early-bird,2,1,16605_2,mid,3,1
4,B10634,3391.07421460378,2,4,2.45365640323609,Sleeping-dog,2,3,10634_2,mid,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...
145,Z13253,3678.8,2,5,2.46367035079282,Big-Screen-lover,2,3,13253_2,mid,1,3
146,Z13534,3662.43752684176,3,6,0.9,Early-bird,3,1,13534_3,mid,3,1
147,Z16428,3516.5,2,5,2.37130050714128,Big-Screen-lover,2,3,16428_2,mid,1,3
148,Z16735,3300.1,2,5,1.70494219968095,Big-Screen-lover,2,2,16735_2,mid,1,2


In [131]:
# 연습문제] CUSTTYPE별로 숫자를 맵핑(벡터화)하여 CUSTTYPE_NO 컬럼을 생성하세요
# CUSTTYPE별 숫자 매핑
labelDict = {
    'Early-bird': 0,
    'Sleeping-dog': 1,
    'Big-Screen-lover': 2
}

# CUSTTYPE_NO 열 생성
customerData['CUSTTYPE_NO'] = customerData['CUSTTYPE'].map( labelDict )

customerData

,CUSTID,AVGPRICE,EMI,DEVICECOUNT,PRODUCTAGE,CUSTTYPE,EMI_NEW,PRODUCTAGE_NEW,NEW_COLUMN,CUST_SEG,CUSTTYPE_NO,PRODUCT_AGE_NEW
0,A13566,5000,3,6,1.67918084803969,Big-Screen-lover,3,2,13566_3,high,2,2
1,A14219,3642.441950493661,2,4,2.68202302093618,Sleeping-dog,2,3,14219_2,mid,1,3
2,A15312,3653.88456537549,2,5,3.2082015713118,Sleeping-dog,2,3,15312_2,mid,1,5
3,A16605,3713.211106711161,2,6,0.9,Early-bird,2,1,16605_2,mid,0,1
4,B10634,3391.07421460378,2,4,2.45365640323609,Sleeping-dog,2,3,10634_2,mid,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...
145,Z13253,3678.8,2,5,2.46367035079282,Big-Screen-lover,2,3,13253_2,mid,2,3
146,Z13534,3662.43752684176,3,6,0.9,Early-bird,3,1,13534_3,mid,0,1
147,Z16428,3516.5,2,5,2.37130050714128,Big-Screen-lover,2,3,16428_2,mid,2,3
148,Z16735,3300.1,2,5,1.70494219968095,Big-Screen-lover,2,2,16735_2,mid,2,2


In [132]:
# 또는 drop_duplicates()를 쓰는 방법도 있다.
# CUSTTYPE별 숫자 매핑을 위한 딕셔너리 생성
custtype_mapping = {custtype: idx for idx, custtype in enumerate(customerData['CUSTTYPE'].unique(), start=1)}

# CUSTTYPE_NO 열 생성
customerData['CUSTTYPE_NO'] = customerData['CUSTTYPE'].map(custtype_mapping)

# 중복 제거
unique_custtype_mapping = customerData[['CUSTTYPE', 'CUSTTYPE_NO']].drop_duplicates().reset_index(drop=True)

print("전체 데이터:")
print(customerData)
print("\n고유한 CUSTTYPE 매핑:")
print(unique_custtype_mapping)

전체 데이터:
     CUSTID           AVGPRICE EMI DEVICECOUNT        PRODUCTAGE  \
0    A13566               5000   3           6  1.67918084803969   
1    A14219  3642.441950493661   2           4  2.68202302093618   
2    A15312   3653.88456537549   2           5   3.2082015713118   
3    A16605  3713.211106711161   2           6               0.9   
4    B10634   3391.07421460378   2           4  2.45365640323609   
..      ...                ...  ..         ...               ...   
145  Z13253             3678.8   2           5  2.46367035079282   
146  Z13534   3662.43752684176   3           6               0.9   
147  Z16428             3516.5   2           5  2.37130050714128   
148  Z16735             3300.1   2           5  1.70494219968095   
149  Z19835   3326.52618288181   2           6               1.2   

             CUSTTYPE EMI_NEW  PRODUCTAGE_NEW NEW_COLUMN CUST_SEG  \
0    Big-Screen-lover       3               2    13566_3     high   
1        Sleeping-dog       2        

In [133]:
# 연습문제] productage가 1보다작으면 1 2보다작으면2 3보다작으면3 이외는 5라고 새로운 컬럼 product_age_new를 생성하세요
# 조건 정의
conditions = [
    (customerData['PRODUCTAGE'].astype(float) < 1),
    (customerData['PRODUCTAGE'].astype(float) < 2),
    (customerData['PRODUCTAGE'].astype(float) < 3)
]

# 각 조건에 해당하는 값 정의
choices = [1, 2, 3]

# 조건에 따라 PRODUCT_AGE_NEW 열 생성
customerData['PRODUCT_AGE_NEW'] = np.select(conditions, choices, default=5)

customerData

,CUSTID,AVGPRICE,EMI,DEVICECOUNT,PRODUCTAGE,CUSTTYPE,EMI_NEW,PRODUCTAGE_NEW,NEW_COLUMN,CUST_SEG,CUSTTYPE_NO,PRODUCT_AGE_NEW
0,A13566,5000,3,6,1.67918084803969,Big-Screen-lover,3,2,13566_3,high,1,2
1,A14219,3642.441950493661,2,4,2.68202302093618,Sleeping-dog,2,3,14219_2,mid,2,3
2,A15312,3653.88456537549,2,5,3.2082015713118,Sleeping-dog,2,3,15312_2,mid,2,5
3,A16605,3713.211106711161,2,6,0.9,Early-bird,2,1,16605_2,mid,3,1
4,B10634,3391.07421460378,2,4,2.45365640323609,Sleeping-dog,2,3,10634_2,mid,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...
145,Z13253,3678.8,2,5,2.46367035079282,Big-Screen-lover,2,3,13253_2,mid,1,3
146,Z13534,3662.43752684176,3,6,0.9,Early-bird,3,1,13534_3,mid,3,1
147,Z16428,3516.5,2,5,2.37130050714128,Big-Screen-lover,2,3,16428_2,mid,1,3
148,Z16735,3300.1,2,5,1.70494219968095,Big-Screen-lover,2,2,16735_2,mid,1,2


In [134]:
# 정렬하기
# 매우 중요. 키를 따로 처리함으로 나중에 유용하게 많이 쓰임.
sortKey = ["EMI"]

In [136]:
# ignore_index= 는 2020년도에 추가된 함수로 꼬인 인덱스번호에 정렬된 인덱스 번호를 다시 부여해줌
sortedCsData = customerData.sort_values(by=sortKey, ascending=False, ignore_index=True)
sortedCsData

,CUSTID,AVGPRICE,EMI,DEVICECOUNT,PRODUCTAGE,CUSTTYPE,EMI_NEW,PRODUCTAGE_NEW,NEW_COLUMN,CUST_SEG,CUSTTYPE_NO,PRODUCT_AGE_NEW
0,A13566,5000,3,6,1.67918084803969,Big-Screen-lover,3,2,13566_3,high,1,2
1,W13404,4296.87374265383,3,6,1.1,Early-bird,3,2,13404_3,high,3,2
2,N13273,3839.69473227858,3,6,1.1,Early-bird,3,2,13273_3,mid,3,2
3,R16364,3592.3535401614504,3,6,0.9,Early-bird,3,1,16364_3,mid,3,1
4,L13892,3263.5151446329305,3,6,1.1,Early-bird,3,2,13892_3,mid,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...
145,K10712,3419.46259594748,1,4,2.8257901416253297,Sleeping-dog,1,3,10712_1,mid,2,3
146,C18622,3246.0,1,5,1.89162412926089,Big-Screen-lover,1,2,18622_1,mid,1,2
147,N14862,2390.08373567855,1,6,1.0,Early-bird,1,2,14862_1,low,3,2
148,M16452,3353.53728971703,1,4,1.94035696275532,Sleeping-dog,1,2,16452_1,mid,2,2


In [137]:
# GroupBy
groupKey = ["EMI"] # 지역별, 상품별 등으로 예측레벨을 변화시키면서 예측모델을 만들 수 있음

In [138]:
# 컬럼들의 데이터타입을 확인
sortedCsData.dtypes

,0
CUSTID,object
AVGPRICE,object
EMI,object
DEVICECOUNT,object
PRODUCTAGE,object
CUSTTYPE,object
EMI_NEW,object
PRODUCTAGE_NEW,int64
NEW_COLUMN,object
CUST_SEG,object


In [139]:
# AVGPRICE 도 object 즉, 전에 문자형으로 바꿨기 때문에 float으로 변환필요
sortedCsData["AVGPRICE"] = sortedCsData.AVGPRICE.astype(float)

In [140]:
groupData = sortedCsData.groupby(by=groupKey)["AVGPRICE"].agg(["mean"]).reset_index()

In [141]:
groupData =  groupData.rename( columns={"mean":"AVG_MEAN"})

In [142]:
# refinedSelloutData 에서 →
# 지역, 상품, 연주차 컬럼순으로 오름차순 정렬하여
# sortedData 변수에 담으세요

In [ ]:
# [지역, 상품, 연도 별 집계]
# sortedData 에서 지역, 상품, 연도 단위
# 판매량(QTY_NEW) 의 평균 연산 후
# groupData 변수에 담으세요
# 이후 컬럼명을 QTY_MEAN로 변경하세요
